.

[목표]

ASIS EMR의 jupyterhub 계정정보와 라이브러리를 백업해서 TOBE EMR에서 restore하는 쉘스크립트 예시

- backup.sh

In [ ]:
#!/bin/bash

####################################################################
# backup pip3 library in EMR master node
####################################################################

pip3 freeze > /home/hadoop/requirements.txt
aws s3 cp /home/hadoop/requirements.txt s3://pms-bucket-test/dev_emr_backup/

####################################################################
# backup jupyterhub user info
####################################################################

echo "c.LocalAuthenticator.create_system_users = True" | sudo tee -a /etc/jupyter/conf/jupyterhub_config.py
sudo docker restart jupyterhub

token=$(sudo docker exec jupyterhub /opt/conda/bin/jupyterhub token jovyan | tail -1)
user_list=$(curl -XGET -s -k https://$(hostname):9443/hub/api/users -H "Authorization: token $token" | jq '.[].name' | sed 's/"//g')
echo $user_list > /home/hadoop/jupyterhub_user_list.txt
aws s3 cp /home/hadoop/jupyterhub_user_list.txt s3://pms-bucket-test/jupyterhub_user_list.txt

####################################################################
# backup jupyterhub pip library
####################################################################

sudo docker exec jupyterhub bash -c "pip freeze > jupyterhub_requirements.txt"
sudo docker cp jupyterhub:/home/jovyan/jupyterhub_requirements.txt /home/hadoop/
aws s3 cp /home/hadoop/jupyterhub_requirements.txt s3://pms-bucket-test/dev_emr_backup/

####################################################################
# backup yum library in EMR master node
####################################################################

rpm -qa > /home/hadoop/yum_list_backup.log
aws s3 cp /home/hadoop/yum_list_backup.log s3://pms-bucket-test/dev_emr_backup/

- restore.sh

In [ ]:
#!/bin/bash

####################################################################
# restore yum library in EMR master node
####################################################################

aws s3 cp s3://pms-bucket-test/dev_emr_backup/yum_list_backup.log /home/hadoop/yum_list_backup.log
sudo yum -y install $(cat /home/hadoop/yum_list_backup.log)

####################################################################
# restore pip3 library in EMR master node
####################################################################

aws s3 cp s3://pms-bucket-test/dev_emr_backup/requirements.txt /home/hadoop/requirements.txt
sudo pip3 install $(grep -ivE "beautifulsoup4|boto|click|jmespath|joblib|lxml|mysqlclient|nltk|nose|numpy|py-dateutil|python37-sagemaker-pyspark|pytz|PyYAML|regex|six|tqdm|windmill" /home/hadoop/requirements.txt)

####################################################################
# restore jupyterhub user info
####################################################################

echo "c.LocalAuthenticator.create_system_users = True" | sudo tee -a /etc/jupyter/conf/jupyterhub_config.py
sudo docker restart jupyterhub

aws s3 cp s3://pms-bucket-test/jupyterhub_user_list.txt /home/hadoop/jupyterhub_user_list.txt

set -x
USERS=($( cat /home/hadoop/jupyterhub_user_list.txt ))
TOKEN=$(sudo docker exec jupyterhub /opt/conda/bin/jupyterhub token jovyan | tail -1)
password=$(echo "bXlwYXNzd2Q=" | base64 -d)
# bXlwYXNzd2Q= : mypasswd

for i in "${USERS[@]}";
do 
   sudo docker exec jupyterhub useradd -m -s /bin/bash -N $i
   sudo docker exec jupyterhub bash -c "echo $i:$password | chpasswd"
done

####################################################################
# restore jupyterhub pip library
####################################################################

aws s3 cp s3://pms-bucket-test/dev_emr_backup/jupyterhub_requirements.txt /home/hadoop/jupyterhub_requirements.txt
sudo docker cp /home/hadoop/jupyterhub_requirements.txt jupyterhub:/home/jovyan/jupyterhub_requirements.txt
sudo docker exec jupyterhub bash -c "pip install -r jupyterhub_requirements.txt"